In [1]:
import os
import numpy as np
import mne
import pandas as pd
import random
from tqdm import tqdm

random_seed = 1
random.seed(random_seed)
np.random.seed(random_seed)

labels_remap = {276: 'idling_eyes_open',
277: 'idling_eyes_closed',
768: 'start_of_a_trial',
769: 'imagine_left', 
770: 'imagine_right', 
771: 'imagine_foot', 
772: 'imagine_tongue', 
783: 'unknown',
1023: 'rejected_trial',
1072: 'eye_movements',
32766: 'start_of_a_new_run'}

def process_annots(annot_df, freq = 250):
    
    timestep = pd.to_timedelta(f"{1 / freq} seconds")
    new_annots = {"timestamp": [], "label": []}
    for onset, duration, description in annot_df.values:
        duration = pd.to_timedelta(f"{duration} seconds")
        stop_onset = onset + duration

        while onset != stop_onset:
            new_annots["timestamp"].append(onset)
            new_annots["label"].append(description)
            onset = onset + timestep

    new_annots = pd.DataFrame(new_annots)
    return new_annots

# Change original labels into interpretable labels
def change_labels(label_df, labels_remap):
    label_df["description"] = label_df["description"].astype(int).replace(labels_remap)
    return label_df

In [2]:
dataset_path = 'dataset/BCICIV_2a_gdf/'
save_path = 'dataset/BCICIV_2a_gdf_csv_full/'

if not os.path.isdir(save_path):
    os.mkdir(save_path)

subjects_idc = [f"A{i:02d}T.gdf" for i in range(1, 10)]


In [3]:
freq = 250
timestep = pd.to_timedelta(f"{1 / freq} seconds")

In [4]:
diff_df = []

In [5]:
for subject_id in tqdm(subjects_idc):
    run_path = os.path.join(dataset_path, subject_id)
    print(run_path)
    raw = mne.io.read_raw_gdf(run_path, verbose=False)
    raw_data = raw.to_data_frame()
    raw_data = raw_data.drop(columns=["time"])
    original_data_length = raw_data.shape[0]

    raw_labels = raw.annotations.to_data_frame()
    raw_labels = change_labels(raw_labels, labels_remap)
    raw_labels = process_annots(raw_labels)

    if raw_data.shape[0] != raw_labels.shape[0]:
        len_diff = raw_data.shape[0] - raw_labels.shape[0]
        
        unique = raw_data.iloc[raw_labels.shape[0]:].values
        unique = np.unique(unique)

        diff_df.append({"filename": run_path, "length_diff": len_diff, "diff_value_unique": unique})
    
    raw_data = pd.concat([raw_labels, raw_data], axis=1)
    raw_data = raw_data.dropna()
    raw_data = raw_data.drop(columns=['EOG-left', 'EOG-central', 'EOG-right'])
    raw_data.to_csv(os.path.join(save_path, subject_id.rstrip(".gdf") + ".csv"), index=False)
    raw_labels.to_csv(os.path.join(save_path, subject_id.rstrip(".gdf") + "_labels.csv"), index=False)

diff_df = pd.DataFrame(diff_df)
diff_df.to_csv(os.path.join(save_path, "data_label_diff.csv"))

  0%|          | 0/9 [00:00<?, ?it/s]/home/amar/Desktop/thesis-2022/env/lib/python3.7/site-packages/mne/io/edf/edf.py:1047: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, UINT8).tolist()[0]
/tmp/ipykernel_12987/508169274.py:4: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_gdf(run_path, verbose=False)


dataset/BCICIV_2a_gdf/A01T.gdf


 11%|█         | 1/9 [00:24<03:14, 24.33s/it]/home/amar/Desktop/thesis-2022/env/lib/python3.7/site-packages/mne/io/edf/edf.py:1047: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, UINT8).tolist()[0]
/tmp/ipykernel_12987/508169274.py:4: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_gdf(run_path, verbose=False)


dataset/BCICIV_2a_gdf/A02T.gdf


 22%|██▏       | 2/9 [00:47<02:46, 23.73s/it]/home/amar/Desktop/thesis-2022/env/lib/python3.7/site-packages/mne/io/edf/edf.py:1047: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, UINT8).tolist()[0]
/tmp/ipykernel_12987/508169274.py:4: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_gdf(run_path, verbose=False)


...

 78%|███████▊  | 7/9 [02:37<00:44, 22.40s/it]/home/amar/Desktop/thesis-2022/env/lib/python3.7/site-packages/mne/io/edf/edf.py:1047: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, UINT8).tolist()[0]
/tmp/ipykernel_12987/508169274.py:4: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_gdf(run_path, verbose=False)


dataset/BCICIV_2a_gdf/A08T.gdf


 89%|████████▉ | 8/9 [02:59<00:22, 22.40s/it]/home/amar/Desktop/thesis-2022/env/lib/python3.7/site-packages/mne/io/edf/edf.py:1047: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, UINT8).tolist()[0]
/tmp/ipykernel_12987/508169274.py:4: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_gdf(run_path, verbose=False)


dataset/BCICIV_2a_gdf/A09T.gdf


100%|██████████| 9/9 [03:22<00:00, 22.50s/it]


In [6]:
filenames = os.listdir('dataset/BCICIV_2a_gdf_csv_full/')
if not os.path.isdir('dataset/BCICIV_2a_gdf_csv_full_imagine/'):
    os.mkdir('dataset/BCICIV_2a_gdf_csv_full_imagine/')

In [7]:
for filename in tqdm(filenames):
    if 'label' in filename:
        continue
    df = pd.read_csv("dataset/BCICIV_2a_gdf_csv_full/" + filename)
    df = df[df["label"].str.contains("imagine")]
    df.to_csv("dataset/BCICIV_2a_gdf_csv_full_imagine/" + filename[:4] + "_imagine.csv", index=False)

100%|██████████| 19/19 [00:47<00:00,  2.49s/it]


In [12]:
df = pd.read_csv('dataset/BCICIV_2a_gdf_csv_full_imagine/A06T_imagine.csv')

In [15]:
df.iloc[:, 2:].shape

(70112, 22)

In [16]:
import pickle

data = pickle.load(open('dataset/train/cross_subject_data_bci_2a_0_5_subjects.pickle', 'rb'))

In [21]:
np.unique(data['y_train'])

array(['imagine_foot', 'imagine_left', 'imagine_right', 'imagine_tongue'],
      dtype=object)

In [23]:
data['X_train'].shape

(383425, 22)

In [24]:
data['X_train'].reshape(-1, 100, 22).shape

ValueError: cannot reshape array of size 8435350 into shape (100,22)